# StationSim - Grand Central Station version
    author: P. Ternes

In this text we track changes made in the StationSim model to emulate the data from the Grand Central Station.

## Manly changes

* Create gates in all sides;
* Create a forbidden region in the center of the station to represent the clock;
* Designate all gates as possible entry and exit points;
* Change the way the model identifies a collision;
* Change the way the the agents perform a wiggle.

## Grand Central Station

You can access the Grand Station data here:
http://www.ee.cuhk.edu.hk/~xgwang/grandcentral.html

The main concourse of the station has a rectangular shape and has gates on all sides. We are considerate one gate on the South Side, two gates on the West Side, five gates on the North Side, and two gates on the East Side. The station also has a clock in the center of the main concourse. A schematic view of the station can be seen in the figure below. The gray region in the figure represent the simulated region.

<img src="figs/station1.png" width="300">


To define the size of the station, the location of the gates and the location and size of the clock, it is necessary to perform a calibration with real data.

## Create Grand Central Station in the StationSim model
To create a station with the structure of the Grand Central station is necesary to pass to the model a parameter with the key 'station' and the value 'Grand_Central'. E.g. like this:

In [ ]:
model_params = {'station': 'Grand_Central'}

If the 'Grand_Central' parameter is passed to the model, the parameters 'width', 'height', 'gates_in' and 'gates_out' will be determined within the set_station method. If any of these parameters are passed to the model, they are just ignored.

The <b>set_station</b> method is:

In [ ]:
    def set_station(self):
        if(self.station == 'Grand_Central'):
            self.width = 200
            self.height = 400
            self.gates_locations =\
                np.array([[0, self.height/2],  # south side
                          [20, self.height], [170, self.height],  # west side
                          [20, 0], [170, 0],  # east side
                          [self.width, 60], [self.width, 125],  # north side
                          [self.width, 200], [self.width, 275],  # north side
                          [self.width, 340]])  # north side
            self.gates_in = len(self.gates_locations)
            self.gates_out = len(self.gates_locations)
            self.clock = Agent(self, self.pop_total)
            self.clock.size = 10.0
            self.clock.location = [self.width/2.0, self.height/2.0]
            self.clock.speed = 0.0
        else:
            self.gates_locations = np.concatenate([
                Model._gates_init(0, self.height, self.gates_in),
                Model._gates_init(self.width, self.height, self.gates_out)])
            # create a clock outside the station.
            self.clock = Agent(self, self.pop_total)
            self.clock.speed = 0.0

            if(self.station is not None):
                warnings.warn(
                    "The station parameter passed to the model is not valid; "
                    "Using the default station.",
                    RuntimeWarning
                )

Note that if a different station name is passed, than the classical structure of the StationSim model is used. This means that only gates at left side are used as entrace points and gates at right side are used as exit points.

To ensure that the code is executed, regardless of the structure of the chosen station, a condition is used to determine the exit gate.

In [ ]:
        self.gate_in = np.random.randint(model.gates_in)
        gate_out = np.random.randint(model.gates_out) + model.gates_in
        while (gate_out == self.gate_in or
               gate_out >= len(model.gates_locations)):
            gate_out = np.random.randint(model.gates_out)
        self.loc_desire = self.set_agent_location(model, gate_out)

## Agent initial and final position
Each agent, when created, needs to receive a start position and a desired end position. These positions are based on the position of the entry and exit gates defined for that agent. To simulate the width of the gates a perturbartion is added to each selected position. In addition, it is necessary to ensure that the agents have positions that are a little more than a body away from the station walls.

The <b>set_agent_location</b> method returns a position based on the mentioned criteria.

In [ ]:
    def set_agent_location(self, model, gate):
        '''
            Define one final or initial position for the agent.

            It is necessary to ensure that the agent has a distance from
            the station wall compatible with its own size.
        '''
        perturb = model.gates_space * np.random.uniform(-10, +10)
        if(model.gates_locations[gate][0] == 0):
            return model.gates_locations[gate] + [1.05*self.size, perturb]
        elif(model.gates_locations[gate][0] == model.width):
            return model.gates_locations[gate] + [-1.05*self.size, perturb]
        elif(model.gates_locations[gate][1] == 0):
            return model.gates_locations[gate] + [perturb, 1.05*self.size]
        else:
            return model.gates_locations[gate] + [perturb, -1.05*self.size]

The initial position also needs an aditional criterion. This position must be different from the current position of any other active agent. This condition is necessary due to the new definition of collision. If two agents are in the same position, then the new collision definition will cause the dynamics in the system to stop.

The <b>activate</b> method creats correctly a initial position. Note that after 10 attempts to create a starting position, if it is impossible to designate a unique position to the agent, than, the agent will be activated only in the next step.

In [ ]:
    def activate(self, model):
        '''
        Test whether an agent should become active.
        This happens when the model time is greater than the agent's
        activate time.

        It is necessary to ensure that the agent has an initial position
        different from the position of all active agents. If it was not
        possible, activate the agent on next time step.
        '''
        if self.status == 0:
            if model.total_time > self.steps_activate:
                state = model.get_state('location2D')
                model.tree = cKDTree(state)
                for _ in range(10):
                    new_location = self.set_agent_location(model, self.gate_in)
                    neighbouring_agents = model.tree.query_ball_point(
                        new_location, self.size*1.1)
                    if (neighbouring_agents == [] or
                            neighbouring_agents == [self.unique_id]):
                        self.location = new_location
                        self.status = 1
                        model.pop_active += 1
                        self.step_start = model.total_time  # model.step_id
                        self.loc_start = self.location
                        break

## New colision definition

In the default version of the StationSim model, the movement of the agents occurs mainly in the horizontal direction, from left to right side. This movement limitation allows the use of a simplified collision definition.

By creating gates on all sides of the station and allowing them to be points of entry and exit, we make it possible for agents to have the most varied directions of movement. Thus, a more robust definition of collision is necessary.

The new definition of colision is obtained through the equation of motion of each agent. Before a colision, the movemment is linear, and can be described by:
$$\vec{r}_i' = \vec{r}_i + \vec{v}_i\Delta t ,$$
where $\vec{r}_i = (x_i, y_i)$ is the position of agent $i$ at time $t$, $\vec{r}_i'= (x_i', y_i')$ is the position of agent i at time $t'$, $\vec{v}_i=(v_{xi}, v_{yi})$ is the agent velocity, and $\Delta t = t'-t$ is the time variation.

### Collision between two agents

The next figure illustrate the colision between two agents.

<img src="figs/collision_scheme.png" width="600">

Note that, when an agent $i$ collide with another agent $j$, the distance between the center of the agents is $\sigma = \sigma_i + \sigma_j$, where $\sigma_i$ and $\sigma_j$ are related with the agents width. 
It is possible to obtain the distance between two agents $i$ and $j$ using their positions:
$$ \Delta r'^{2} = (x_j' - x_i')^2 + (y_j' - y_i')^2 ,$$
therefore, in a collision we have $\Delta r'^2 = \sigma^2$. Putting all the equations together and solving the quadratic equation, it is possible to find the time variation between the beginning of the movement and the collision.

$$\Delta t = \left\{ \begin{array}{lcl}
\infty & \mbox{if} & \Delta\vec{v}\cdot\Delta\vec{r}\ge0, \\ 
\infty & \mbox{if} & d < 0, \\
\dfrac{-\Delta\vec{v}\cdot\Delta\vec{r}-\sqrt{d}}{\Delta\vec{v}\cdot\Delta\vec{v}} & \mbox{if} & \mbox{otherwise,}
\end{array}\right.$$

where
$$\begin{array}{l}
d = (\Delta\vec{v}\cdot\Delta\vec{r})^2 - (\Delta\vec{v}\cdot\Delta\vec{v})(\Delta\vec{r}\cdot\Delta\vec{r} - \sigma^2),\\ 
\Delta\vec{v} = \vec{v}_j - \vec{v}_i = (v_{xj}-v_{xi}, v_{yj}-v_{yi}),\\
\Delta\vec{r} = \vec{r}_j - \vec{r}_i = (r_{xj}-r_{xi}, r_{yj}-r{yi}).\\
\end{array}$$

When $\Delta\vec{v}\cdot\Delta\vec{r}\ge 0 $ or $d < 0$, the agents do not  collide, even though the distance between them is $\sigma$. This situation can occur if their movemment are parrallel or in opposite directions.

The <b>get_collisionTime2Agents</b> method return the collision time between two agents.


In [ ]:
    def get_collisionTime2Agents(self, agentB):
        '''
        Returns the collision time between two agents.
        '''
        tmin = 1.0e300

        rAB = self.location - agentB.location
        directionA = self.get_direction(self.loc_desire, self.location)
        directionB = agentB.get_direction(agentB.loc_desire, agentB.location)
        sizeAB = self.size + agentB.size

        vA = self.speed
        vB = agentB.speed
        vAB = vA*directionA - vB*directionB
        bAB = np.dot(vAB, rAB)
        if bAB < 0.0:
            delta = bAB**2 - (np.dot(vAB, vAB)*(np.dot(rAB, rAB) - sizeAB**2))
            if (delta > 0.0):
                collisionTime = abs((-bAB - np.sqrt(delta)) / np.dot(vAB, vAB))
                tmin = collisionTime

        return tmin

### Collision between an agent and a wall

In addition to colliding with another agent, an agent can also collide with a wall.
The next figure illustrate the colision between an agent and a wall.

<img src="figs/collision-wall.png" width="350">

Note that, when an agent $i$ collide with a wall, the distance between the center of the agent and the wall is $\sigma_i$, where $\sigma_i$ is related with the agent width. Considering that the station has a retangular shape, the agent can collide with four different walls. The equations to determine the colision time for each possible wall are:
$$\begin{array}{c@{}c@{}c@{}}
\mbox{Vertical wall} & \hspace{50pt} & \mbox{Horizontal wall} \\
\Delta t = \left\{ \begin{array}{lcl}
    (\sigma_i -  x_i)/v_{xi} & \mbox{if} & v_{xi} < 0; \\ 
(w - \sigma_i -  x_i)/v_{xi} & \mbox{if} & v_{xi} > 0; \\                            
\infty & \mbox{if} & v_{xi} = 0;
\end{array}\right. & ~ & \Delta t = \left\{ \begin{array}{lcl}
    (\sigma_i -  y_i)/v_{yi} & \mbox{if} & v_{yi} < 0; \\ 
(h - \sigma_i -  y_i)/v_{yi} & \mbox{if} & v_{yi} > 0; \\                            
\infty & \mbox{if} & v_{yi} = 0;
\end{array}\right.
\end{array}$$
where, $w$ is the station width and $h$ is the station heigh.

The minor collision time between an agent and a wall is determined by the <b>get_collisionTimeWall</b> method.

In [ ]:
    def get_collisionTimeWall(self, model):
        '''
        Returns the shortest collision time between an agent and a wall.
        '''
        tmin = 1.0e300
        collisionTime = 1.0e300

        direction = self.get_direction(self.loc_desire, self.location)
        vx = self.speed*direction[0]  # horizontal velocity
        vy = self.speed*direction[1]  # vertical velocity

        if(vy > 0):  # collision in botton wall
            collisionTime = (model.height - self.size - self.location[1]) / vy
        elif (vy < 0):  # collision in top wall
            collisionTime = (self.size - self.location[1]) / vy
        if (collisionTime < tmin):
            tmin = collisionTime
        if(vx > 0):  # collision in right wall
            collisionTime = (model.width - self.size - self.location[0]) / vx
        elif (vx < 0):  # collision in left wall
            collisionTime = (self.size - self.location[0]) / vx
        if (collisionTime < tmin):
            tmin = collisionTime

        return tmin

### Velocity variation before wiggle

In the default version of the StationSim model when a collision was identified, before wiggle, the agent's velocity was decreased to try avoid the collision.

Unfortunately, by changing the collision definition, this feature was lost.

## Step implementation

In this new version of StationSim, we check if a collision occurs before moving agents. At the beginning of each model step, a table is created containing the time collision for each possible collision, including collisions with the wall, with other active agents and with the station clock.

Using the collision table, the shortest collision time is selected. The <b>get_collisionTable</b> method is:

In [ ]:
    def get_collisionTable(self):
        '''
        Returns the time of next colision (tmin) and a table with 
        information about every possible colision:
        - collisionTable[0]: collision time
        - collisionTable[1]: agent agent.unique_id
        '''
        collisionTable = []
        for i in range(self.pop_total):
            if (self.agents[i].status == 1):
                collisionTime = self.agents[i].get_collisionTimeWall(self)
                collision = (collisionTime, i)
                collisionTable.append(collision)

                collisionTime =\
                    self.agents[i].get_collisionTime2Agents(self.clock)
                collision = (collisionTime, i)
                collisionTable.append(collision)

                for j in range(i+1, self.pop_total):
                    if (self.agents[j].status == 1):
                        collisionTime = self.agents[i].\
                            get_collisionTime2Agents(self.agents[j])
                        collision = (collisionTime, i)
                        collisionTable.append(collision)
                        collision = (collisionTime, j)
                        collisionTable.append(collision)

        try:
            tmin = min(collisionTable)
            tmin = tmin[0]
        except:
            tmin = 1.0e300

        return collisionTable, tmin

Therefore, after call the <b>get_collisionTable</b> method we have the information about when the next collision will occur. If the next collision occurs in a time greater than 1, all the active agent will be moved in a straight line for 1 step. 

If the next collision occurs in a time lower than 1, all active agents will be moved in a straight line by a time equal to the next collision time. It is important to remember that this is a multibody problem, so it is possible to have simultaneous collisions. To track all possible collisions, a wiggle table is created with all colisions that occur at the same current collision time (inside a tolerance interval time). The <b>get_wiggleTable</b> is:

In [ ]:
    def get_wiggleTable(self, collisionTable, time):
        '''
        Returns a table with the agent.unique_id of all agents that
        collide in the specified time. A tolerance time is used to
        capture almost simultaneous collisions.

        Each line in the collisionTable has 2 columns:
        - Column 0: collision time
        - Column 1: agent.unique_id
        '''
        return set([line[1] for line in collisionTable
                    if (abs(line[0] - time) < self.tolerance)])

All agents in the wiggle table will be passed to <b>set_wiggle</b> method where the agents that are colliding were moved on a normal direction. It is very important to ensure that after wiggle the agent is not above any other agent. If after 10 attempts to move the agent it was impossible to find a new location without another agent, the agent just remain stopped. The <b>set_wiggle</b> method is:

In [ ]:
    def set_wiggle(self, model):
        '''
        Determine a new position for an agent that collided with another
        agent, or with some element of the station.
        The new position simulates a lateral step. The side on which the
        agent will take the step is chosen at random, as well as the
        amplitude of the step.

        Description:
        - Determine a new position and check if it is a unique position.
        - If it is unique, then the agent receives this position.
        - Otherwise, a new position will be determined.
        - This process has a limit of 10 attempts. If it is not possible
        to determine a new unique position, the agent just stay stopped.
        '''
        direction = self.get_direction(self.loc_desire, self.location)

        state = model.get_state('location2D')
        model.tree = cKDTree(state)
        for _ in range(10):
            normal_direction = self.get_normal_direction(direction)
            new_location = self.location +\
                normal_direction *\
                np.random.normal(self.size, self.size/2.0)

            # Rebound
            if not model.is_within_bounds(self, new_location):
                new_location = model.re_bound(self, new_location)

            # collision_map
            if model.do_history:
                self.history_collisions += 1
                model.history_collision_locs.append(new_location)
                model.history_collision_times.append(model.total_time)

            # Check if the new location is possible
            neighbouring_agents = model.tree.query_ball_point(new_location,
                                                              self.size*1.1)
            dist = self.distance(new_location, model.clock.location)
            if ((neighbouring_agents == [] or
                    neighbouring_agents == [self.unique_id]) and
                    (dist > (self.size + model.clock.size))):
                self.location = new_location

                # wiggle_map
                if model.do_history:
                    self.history_wiggles += 1
                    model.history_wiggle_locs.append(new_location)
                break

## Deleted code

As the defition of colision changed, some parts of the code were completely changed or deleted. Here is a summary of the main changes made:

* Agent.move: this method was completely replaced for a new Agent.move method;
* Agent.collision: deleted;
* Agent.neighbourhood: deleted;
* Agent.wiggle and model.max_wiggle parameters: deleted. The new wiggle is related with the agent size to simulate a real human step.

## Preliminar results

Both, classical and Grand Central versions, are working with the new collision definition. The basic experiments for this model can be found at [`gcs_experiments`](experiments/gcs_experiments/gcs_experiments.ipynb).

## Grand Central station calibration

The next step is to use the real data from the Grand Central station to calibrate some parameters. The main idea is to obtain general information from the real system so that the simulation parameters have values consistent with reality. The main parameters are:

* Characteristics of GC station: station size, clock location and size, gates location;
* Characteristics of agents: agent size, agent mean velocity;
* Flow rate of agents through the gates to adjust the activation rate;
